In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
cpp_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/extracted_data/cpp_df.csv'
ptio2_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/extracted_data/ptio2_df.csv'
temperature_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/extracted_data/temperature_df.csv'
lpr_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/extracted_data/lpr_df.csv'
hr_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/extracted_data/hr_df.csv'
drug_administration_path = '/Users/jk1/stroke_datasets/ptiO2-Studie/drug_administrations.xlsx'

In [ ]:
exclude_short_infusions = True

In [ ]:
cpp_df = pd.read_csv(cpp_path)
ptio2_df = pd.read_csv(ptio2_path)
temperature_df = pd.read_csv(temperature_path)
lpr_df = pd.read_csv(lpr_path)
hr_df = pd.read_csv(hr_path)
drug_administration_df = pd.read_excel(drug_administration_path)

# Data Exploration

In [ ]:
drug_administration_df = drug_administration_df[drug_administration_df.monitored]
# exclude if further_exclusion_criterium is not Nan
drug_administration_df = drug_administration_df[pd.isna(drug_administration_df['further_exclusion_criterium'])]

In [ ]:
if exclude_short_infusions:
    drug_administration_df['infusion_duration'] = (pd.to_datetime(drug_administration_df['drug_end']) - pd.to_datetime(drug_administration_df['drug_start'])).dt.total_seconds() / 3600
    print(f'Excluding {drug_administration_df[drug_administration_df["infusion_duration"] <= 1].shape[0]} infusions with duration <= 1h')
    drug_administration_df = drug_administration_df[drug_administration_df['infusion_duration'] > 1]

In [ ]:
drug_administration_df.pat_nr.nunique(), drug_administration_df.shape

In [ ]:
for var_df in [ptio2_df, cpp_df, temperature_df, lpr_df, hr_df]:
    var_df['datetime'] = pd.to_datetime(var_df['datetime'])


In [ ]:
# for every drug administration extract data from -xh to +xh around start
time_window = 12

associated_ptio2_df = pd.DataFrame()
associated_cpp_df = pd.DataFrame()
associated_temperature_df = pd.DataFrame()
associated_hr_df = pd.DataFrame()
associated_lpr_df = pd.DataFrame()
associated_ci_df = pd.DataFrame()

for index, row in drug_administration_df.iterrows():
    lower_bound = row['drug_start'] - pd.to_timedelta(time_window, unit='h')
    upper_bound = row['drug_start'] + pd.to_timedelta(time_window, unit='h')
    instance_associated_ptio2_df = ptio2_df[(ptio2_df['pat_nr'] == row['pat_nr'])
                                            & (ptio2_df['datetime'] >= lower_bound) 
                                            & (ptio2_df['datetime'] <= upper_bound)]
    instance_associated_ptio2_df['drug_start'] = row['drug_start']
    instance_associated_ptio2_df['relative_datetime'] = (instance_associated_ptio2_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_ptio2_df = pd.concat([associated_ptio2_df, instance_associated_ptio2_df])

    instance_associated_cpp_df = cpp_df[(cpp_df['pat_nr'] == row['pat_nr'])
                                        & (cpp_df['datetime'] >= lower_bound) 
                                        & (cpp_df['datetime'] <= upper_bound)]  
    instance_associated_cpp_df['drug_start'] = row['drug_start']
    instance_associated_cpp_df['relative_datetime'] = (instance_associated_cpp_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_cpp_df = pd.concat([associated_cpp_df, instance_associated_cpp_df])

    instance_associated_temperature_df = temperature_df[(temperature_df['pat_nr'] == row['pat_nr'])
                                        & (temperature_df['datetime'] >= lower_bound)
                                        & (temperature_df['datetime'] <= upper_bound)]
    instance_associated_temperature_df['drug_start'] = row['drug_start']
    instance_associated_temperature_df['relative_datetime'] = (instance_associated_temperature_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_temperature_df = pd.concat([associated_temperature_df, instance_associated_temperature_df])

    instance_associated_hr_df = hr_df[(hr_df['pat_nr'] == row['pat_nr'])
                                        & (hr_df['datetime'] >= lower_bound)   
                                        & (hr_df['datetime'] <= upper_bound)]
    instance_associated_hr_df['drug_start'] = row['drug_start']
    instance_associated_hr_df['relative_datetime'] = (instance_associated_hr_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_hr_df = pd.concat([associated_hr_df, instance_associated_hr_df])

    instance_associated_lpr_df = lpr_df[(lpr_df['pat_nr'] == row['pat_nr'])
                                        & (lpr_df['datetime'] >= lower_bound)
                                        & (lpr_df['datetime'] <= upper_bound)]
    instance_associated_lpr_df['drug_start'] = row['drug_start']
    instance_associated_lpr_df['relative_datetime'] = (instance_associated_lpr_df['datetime'] - row['drug_start']).dt.total_seconds() / 3600
    associated_lpr_df = pd.concat([associated_lpr_df, instance_associated_lpr_df])

    

# Data Cleaning

In [ ]:
allowed_ptio2_range = [0, 200]
allowed_cpp_range = [0, 200]
allowed_temperature_range = [30, 45]
allowed_hr_range = [0, 300]
allowed_lpr_range = [0, 100]

In [ ]:
# drop values outside of allowed range
associated_ptio2_df = associated_ptio2_df[(associated_ptio2_df['ptio2'] >= allowed_ptio2_range[0]) & (associated_ptio2_df['ptio2'] <= allowed_ptio2_range[1])]
associated_cpp_df = associated_cpp_df[(associated_cpp_df['cpp'] >= allowed_cpp_range[0]) & (associated_cpp_df['cpp'] <= allowed_cpp_range[1])]
associated_temperature_df = associated_temperature_df[(associated_temperature_df['temperature'] >= allowed_temperature_range[0]) & (associated_temperature_df['temperature'] <= allowed_temperature_range[1])]
associated_hr_df = associated_hr_df[(associated_hr_df['hr'] >= allowed_hr_range[0]) & (associated_hr_df['hr'] <= allowed_hr_range[1])]
associated_lpr_df = associated_lpr_df[(associated_lpr_df['lpr'] >= allowed_lpr_range[0]) & (associated_lpr_df['lpr'] <= allowed_lpr_range[1])]

# Data Plotting

In [ ]:
associated_ptio2_df['rounded_relative_datetime'] = associated_ptio2_df['relative_datetime'].round(1)
associated_cpp_df['rounded_relative_datetime'] = associated_cpp_df['relative_datetime'].round(1)
associated_temperature_df['rounded_relative_datetime'] = associated_temperature_df['relative_datetime'].round(1)
associated_hr_df['rounded_relative_datetime'] = associated_hr_df['relative_datetime'].round(1)
associated_lpr_df['rounded_relative_datetime'] = associated_lpr_df['relative_datetime'].round(1)

In [ ]:
associated_lpr_df.pat_nr.nunique()

In [ ]:
sns.lineplot(x='rounded_relative_datetime', y='lpr', data=associated_lpr_df.reset_index())

In [ ]:
# plot a line plot of ptio2 and cpp values over relative time
plot_temperature = True
plot_hr = True
plot_lpr = False

label_fontsize = 18
legend_fontsize = 16

fig, ax = plt.subplots(1, 1, figsize=(20, 10))
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=associated_ptio2_df, ax=ax, 
             label=f'PtiO2 (n={associated_ptio2_df.pat_nr.nunique()})', legend=False)
# create second y axis for cpp values
ax2 = ax.twinx()
sns.lineplot(x='rounded_relative_datetime', y='cpp', data=associated_cpp_df, ax=ax2, color='magenta', 
             label=f'CPP (n={associated_cpp_df.pat_nr.nunique()})', legend=False)

if plot_temperature:
    ax3 = ax.twinx()
    sns.lineplot(x='rounded_relative_datetime', y='temperature', data=associated_temperature_df, ax=ax3, color='green', 
                 label=f'Temperature (n={associated_temperature_df.pat_nr.nunique()})', legend=False)
    ax3.set_ylabel('Temperature (°C)', fontsize=label_fontsize)
    ax3.grid(None)
    ax3.spines['right'].set_position(('outward', 60))
    
if plot_hr:
    ax4 = ax.twinx()
    sns.lineplot(x='rounded_relative_datetime', y='hr', data=associated_hr_df, ax=ax4, color='#7b002c', 
                 label=f'HR (n={associated_hr_df.pat_nr.nunique()})', legend=False)
    ax4.set_ylabel('HR (bpm)', fontsize=label_fontsize)
    ax4.grid(None)
    ax4.spines['right'].set_position(('outward', 120)) 

if plot_lpr:
    ax5 = ax.twinx()
    sns.lineplot(x='rounded_relative_datetime', y='lpr', data=associated_lpr_df.reset_index(), ax=ax5, color='orange', 
                 label=f'LPR (n={associated_lpr_df.pat_nr.nunique()})', legend=False)
    ax5.set_ylabel('LPR', fontsize=label_fontsize)
    ax5.grid(None)
    ax5.spines['right'].set_position(('outward', 180))


ax.set_xlabel('Relative time from start of infusion (h)', fontsize=label_fontsize)
ax.set_ylabel('PtiO2 (mmHg)', fontsize=label_fontsize)
ax2.set_ylabel('CPP (mmHg)', fontsize=label_fontsize)

ax2.set_ylim(60,100)
ax2.grid(None)

# show legend
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
legend_lines = lines + lines2
legend_labels = labels + labels2

if plot_temperature:
    lines3, labels3 = ax3.get_legend_handles_labels()
    legend_lines += lines3
    legend_labels += labels3
    
if plot_hr:
    lines4, labels4 = ax4.get_legend_handles_labels()
    legend_lines += lines4
    legend_labels += labels4
    
if plot_lpr:
    lines5, labels5 = ax5.get_legend_handles_labels()
    legend_lines += lines5
    legend_labels += labels5

ax.legend(legend_lines, legend_labels, loc='upper right', fontsize=legend_fontsize)

# # add vertical line at x=0 with text label with "Diclofenac administration"
# ax.axvline(x=0, ymin=0.015, ymax=0.1, color='black', linestyle='--', linewidth=0.7)
# ax.text(0.5, 0.12, 'Start of infusion', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)



In [ ]:
# fig.savefig('/Users/jk1/Downloads/ptio2_cpp_temp_hr_plot.png', dpi=600, bbox_inches='tight')

In [ ]:
# boxplot pre / post ptio2
associated_ptio2_df['pre_post'] = 'pre'
associated_ptio2_df.loc[associated_ptio2_df['relative_datetime'] > 0, 'pre_post'] = 'post'
sns.boxplot(x='pre_post', y='ptio2', hue='pre_post', data=associated_ptio2_df)


In [ ]:
# compare pre / post ptio2 with mixed effect model
import statsmodels.api as sm
import statsmodels.formula.api as smf

mixed_model = smf.mixedlm("ptio2 ~ pre_post", associated_ptio2_df, groups=associated_ptio2_df['pat_nr'])

In [ ]:
mdf = mixed_model.fit()
# print formula
print(mdf.model.formula)
print(mdf.summary())

# compare pre / post temperature

In [ ]:
# compare pre / post temperature
associated_temperature_df['pre_post'] = 'pre'
associated_temperature_df.loc[associated_temperature_df['relative_datetime'] > 0, 'pre_post'] = 'post'
sns.boxplot(x='pre_post', y='temperature', hue='pre_post', data=associated_temperature_df)

In [ ]:
associated_temperature_df[associated_temperature_df['pre_post'] == 'pre']['temperature'].median(), associated_temperature_df[associated_temperature_df['pre_post'] == 'post']['temperature'].median()

## alternative comparator 
use time period with Temp < 37.5 as pre

In [ ]:
temperature_df.head()

In [ ]:
temperature_limit = 37.5
possibilities = [[-24, -12], [-36, -24], [-48, -36], [-60, -48], [-72, -60]]
missing_periods = []

pre_associated_ptio2_df = pd.DataFrame()
pre_associated_cpp_df = pd.DataFrame()
pre_associated_temperature_df = pd.DataFrame()
pre_associated_hr_df = pd.DataFrame()
pre_associated_lpr_df = pd.DataFrame()


for index, row in drug_administration_df.iterrows():
    # find a continuous time period with temperature < 37.5 before drug administration
    period_found = False
    for pos in possibilities:
        lower_bound = row['drug_start'] + pd.to_timedelta(pos[0], unit='h')
        upper_bound = row['drug_start'] + pd.to_timedelta(pos[1], unit='h')
        instance_associated_temperature_df = temperature_df[(temperature_df['pat_nr'] == row['pat_nr'])
                                            & (temperature_df['datetime'] >= lower_bound)
                                            & (temperature_df['datetime'] <= upper_bound)]
        if instance_associated_temperature_df['temperature'].max() < temperature_limit:
            period_found = True
            break
            
    if not period_found:
        missing_periods.append([row['pat_nr'], row['drug_start']])
        continue
    
    pre_instance_associated_ptio2_df = ptio2_df[(ptio2_df['pat_nr'] == row['pat_nr'])
                                            & (ptio2_df['datetime'] >= lower_bound) 
                                            & (ptio2_df['datetime'] <= upper_bound)]
    pre_instance_associated_ptio2_df['drug_start'] = row['drug_start']
    pre_instance_associated_ptio2_df['relative_datetime'] = (pre_instance_associated_ptio2_df['datetime'] - row['drug_start'] - pd.to_timedelta(pos[0], unit='h')).dt.total_seconds() / 3600
    pre_associated_ptio2_df = pd.concat([pre_associated_ptio2_df, pre_instance_associated_ptio2_df])

    pre_instance_associated_cpp_df = cpp_df[(cpp_df['pat_nr'] == row['pat_nr'])
                                        & (cpp_df['datetime'] >= lower_bound) 
                                        & (cpp_df['datetime'] <= upper_bound)]
    pre_instance_associated_cpp_df['drug_start'] = row['drug_start']
    pre_instance_associated_cpp_df['relative_datetime'] = (pre_instance_associated_cpp_df['datetime'] - row['drug_start'] - pd.to_timedelta(pos[0], unit='h')).dt.total_seconds() / 3600
    pre_associated_cpp_df = pd.concat([pre_associated_cpp_df, pre_instance_associated_cpp_df])
    
    pre_instance_associated_temperature_df = temperature_df[(temperature_df['pat_nr'] == row['pat_nr'])
                                        & (temperature_df['datetime'] >= lower_bound)
                                        & (temperature_df['datetime'] <= upper_bound)]
    pre_instance_associated_temperature_df['drug_start'] = row['drug_start']
    pre_instance_associated_temperature_df['relative_datetime'] = (pre_instance_associated_temperature_df['datetime'] - row['drug_start']
                                                                    - pd.to_timedelta(pos[0], unit='h')).dt.total_seconds() / 3600
    pre_associated_temperature_df = pd.concat([pre_associated_temperature_df, pre_instance_associated_temperature_df])
    
    pre_instance_associated_hr_df = hr_df[(hr_df['pat_nr'] == row['pat_nr'])
                                        & (hr_df['datetime'] >= lower_bound)   
                                        & (hr_df['datetime'] <= upper_bound)]
    pre_instance_associated_hr_df['drug_start'] = row['drug_start']
    pre_instance_associated_hr_df['relative_datetime'] = (pre_instance_associated_hr_df['datetime'] - row['drug_start']
                                                            - pd.to_timedelta(pos[0], unit='h')).dt.total_seconds() / 3600
    pre_associated_hr_df = pd.concat([pre_associated_hr_df, pre_instance_associated_hr_df])
    
    pre_instance_associated_lpr_df = lpr_df[(lpr_df['pat_nr'] == row['pat_nr'])
                                        & (lpr_df['datetime'] >= lower_bound)
                                        & (lpr_df['datetime'] <= upper_bound)]
    pre_instance_associated_lpr_df['drug_start'] = row['drug_start']
    pre_instance_associated_lpr_df['relative_datetime'] = (pre_instance_associated_lpr_df['datetime'] - row['drug_start']
                                                            - pd.to_timedelta(pos[0], unit='h')).dt.total_seconds() / 3600
    pre_associated_lpr_df = pd.concat([pre_associated_lpr_df, pre_instance_associated_lpr_df])
    
    
    
        

In [ ]:
allowed_ptio2_range = [0, 200]
allowed_cpp_range = [0, 200]
allowed_temperature_range = [30, 45]
allowed_hr_range = [0, 300]
allowed_lpr_range = [0, 100]

In [ ]:
# drop values outside of allowed range
pre_associated_ptio2_df = pre_associated_ptio2_df[(pre_associated_ptio2_df['ptio2'] >= allowed_ptio2_range[0]) & (pre_associated_ptio2_df['ptio2'] <= allowed_ptio2_range[1])]
pre_associated_cpp_df = pre_associated_cpp_df[(pre_associated_cpp_df['cpp'] >= allowed_cpp_range[0]) & (pre_associated_cpp_df['cpp'] <= allowed_cpp_range[1])]
pre_associated_temperature_df = pre_associated_temperature_df[(pre_associated_temperature_df['temperature'] >= allowed_temperature_range[0]) & (pre_associated_temperature_df['temperature'] <= allowed_temperature_range[1])]
pre_associated_hr_df = pre_associated_hr_df[(pre_associated_hr_df['hr'] >= allowed_hr_range[0]) & (pre_associated_hr_df['hr'] <= allowed_hr_range[1])]
pre_associated_lpr_df = pre_associated_lpr_df[(pre_associated_lpr_df['lpr'] >= allowed_lpr_range[0]) & (pre_associated_lpr_df['lpr'] <= allowed_lpr_range[1])]

In [ ]:
pre_associated_ptio2_df['rounded_relative_datetime'] = pre_associated_ptio2_df['relative_datetime'].round(1)
pre_associated_cpp_df['rounded_relative_datetime'] = pre_associated_cpp_df['relative_datetime'].round(1)
pre_associated_temperature_df['rounded_relative_datetime'] = pre_associated_temperature_df['relative_datetime'].round(1)
pre_associated_hr_df['rounded_relative_datetime'] = pre_associated_hr_df['relative_datetime'].round(1)
pre_associated_lpr_df['rounded_relative_datetime'] = pre_associated_lpr_df['relative_datetime'].round(1)

In [ ]:
len(missing_periods)

In [ ]:
post_associated_ptio2_df = associated_ptio2_df[associated_ptio2_df['relative_datetime'] > 0]
post_associated_cpp_df = associated_cpp_df[associated_cpp_df['relative_datetime'] > 0]
post_associated_temperature_df = associated_temperature_df[associated_temperature_df['relative_datetime'] > 0]
post_associated_hr_df = associated_hr_df[associated_hr_df['relative_datetime'] > 0]
post_associated_lpr_df = associated_lpr_df[associated_lpr_df['relative_datetime'] > 0]

In [ ]:
# plot pre and post in two subplots
plot_temperature = True
plot_hr = True
plot_lpr = False

fig, ax = plt.subplots(2, 1, figsize=(20, 20))
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=pre_associated_ptio2_df, ax=ax[0],
             label=f'PtiO2 (n={pre_associated_ptio2_df.pat_nr.nunique()})', legend=False)
# create second y axis for cpp values
ax2 = ax[0].twinx()
sns.lineplot(x='rounded_relative_datetime', y='cpp', data=pre_associated_cpp_df, ax=ax2, color='magenta',
             label=f'CPP (n={pre_associated_cpp_df.pat_nr.nunique()})', legend=False)

if plot_temperature:
    ax3 = ax[0].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='temperature', data=pre_associated_temperature_df, ax=ax3, color='green',
                 label=f'Temperature (n={pre_associated_temperature_df.pat_nr.nunique()})', legend=False)
    ax3.set_ylabel('Temperature (°C)')
    ax3.grid(None)
    ax3.spines['right'].set_position(('outward', 60))
    
if plot_hr:
    ax4 = ax[0].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='hr', data=pre_associated_hr_df, ax=ax4, color='#7b002c',
                 label=f'HR (n={pre_associated_hr_df.pat_nr.nunique()})', legend=False)
    ax4.set_ylabel('HR (bpm)')
    ax4.grid(None)
    ax4.spines['right'].set_position(('outward', 120))
    
if plot_lpr:
    ax5 = ax[0].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='lpr', data=pre_associated_lpr_df.reset_index(), ax=ax5, color='orange',
                 label=f'LPR (n={pre_associated_lpr_df.pat_nr.nunique()})', legend=False)
    ax5.set_ylabel('LPR')
    ax5.grid(None)
    ax5.spines['right'].set_position(('outward', 180))
    
ax[0].set_xlabel('Relative time from administration (h)')
ax[0].set_ylabel('PtiO2 (mmHg)')
ax2.set_ylabel('CPP (mmHg)')
ax2.set_ylim(60, 100)
ax2.grid(None)

# show legend
lines, labels = ax[0].get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
legend_lines = lines + lines2
legend_labels = labels + labels2

if plot_temperature:
    lines3, labels3 = ax3.get_legend_handles_labels()
    legend_lines += lines3
    legend_labels += labels3
    
if plot_hr:
    lines4, labels4 = ax4.get_legend_handles_labels()
    legend_lines += lines4
    legend_labels += labels4
    
if plot_lpr:
    lines5, labels5 = ax5.get_legend_handles_labels()
    legend_lines += lines5
    legend_labels += labels5
    
ax[0].legend(legend_lines, legend_labels, loc='upper right')

# post
sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=post_associated_ptio2_df, ax=ax[1],
             label=f'PtiO2 (n={post_associated_ptio2_df.pat_nr.nunique()})', legend=False)
# create second y axis for cpp values
ax2 = ax[1].twinx()
sns.lineplot(x='rounded_relative_datetime', y='cpp', data=post_associated_cpp_df, ax=ax2, color='magenta',
             label=f'CPP (n={post_associated_cpp_df.pat_nr.nunique()})', legend=False)


if plot_temperature:
    ax3 = ax[1].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='temperature', data=post_associated_temperature_df, ax=ax3, color='green',
                 label=f'Temperature (n={post_associated_temperature_df.pat_nr.nunique()})', legend=False)
    ax3.set_ylabel('Temperature (°C)')
    ax3.grid(None)
    ax3.spines['right'].set_position(('outward', 60))
    
if plot_hr:
    ax4 = ax[1].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='hr', data=post_associated_hr_df, ax=ax4, color='#7b002c',
                 label=f'HR (n={post_associated_hr_df.pat_nr.nunique()})', legend=False)
    ax4.set_ylabel('HR (bpm)')
    ax4.grid(None)
    ax4.spines['right'].set_position(('outward', 120))
    
if plot_lpr:
    ax5 = ax[1].twinx()
    sns.lineplot(x='rounded_relative_datetime', y='lpr', data=post_associated_lpr_df.reset_index(), ax=ax5, color='orange',
                 label=f'LPR (n={post_associated_lpr_df.pat_nr.nunique()})', legend=False)
    ax5.set_ylabel('LPR')
    ax5.grid(None)
    ax5.spines['right'].set_position(('outward', 180))
    
ax[1].set_xlabel('Relative time from administration (h)')
ax[1].set_ylabel('PtiO2 (mmHg)')
ax2.set_ylabel('CPP (mmHg)')
ax2.set_ylim(60, 100)
ax2.grid(None)

# show legend
lines, labels = ax[1].get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
legend_lines = lines + lines2
legend_labels = labels + labels2

if plot_temperature:
    lines3, labels3 = ax3.get_legend_handles_labels()
    legend_lines += lines3
    legend_labels += labels3
    
if plot_hr:
    lines4, labels4 = ax4.get_legend_handles_labels()
    legend_lines += lines4
    legend_labels += labels4
    
if plot_lpr:
    lines5, labels5 = ax5.get_legend_handles_labels()
    legend_lines += lines5
    legend_labels += labels5
    
ax[1].legend(legend_lines, legend_labels, loc='upper right')

ax[0].set_title('Pre administration (afebrile)')

ax[1].set_title('Post administration')

In [ ]:
# fig.savefig('/Users/jk1/Downloads/pre_post_ptio2_cpp_temp_hr_lpr_plot.png', dpi=300, bbox_inches='tight')

In [ ]:
pre_associated_ptio2_df['pre_post'] = 'pre'
post_associated_ptio2_df['pre_post'] = 'post'
combined_pre_post_ptio2_df = pd.concat([pre_associated_ptio2_df, post_associated_ptio2_df])

In [ ]:
# boxplot pre / post ptio2
sns.boxplot(x='pre_post', y='ptio2', hue='pre_post', data=combined_pre_post_ptio2_df)

In [ ]:
pre_mixed_model = smf.mixedlm("ptio2 ~ pre_post", combined_pre_post_ptio2_df, groups=combined_pre_post_ptio2_df['pat_nr'])
pre_mdf = pre_mixed_model.fit()
print(pre_mdf.summary())

plot every single patient

In [ ]:
for administration_row in drug_administration_df.iterrows():
    
    patient_ptio2_df = associated_ptio2_df[(associated_ptio2_df['pat_nr'] == administration_row[1]['pat_nr'])
                                        & (associated_ptio2_df['drug_start'] == administration_row[1]['drug_start'])]
    patient_cpp_df = associated_cpp_df[(associated_cpp_df['pat_nr'] == administration_row[1]['pat_nr'])
                                        & (associated_cpp_df['drug_start'] == administration_row[1]['drug_start'])]
    if patient_ptio2_df.shape[0] == 0 or patient_cpp_df.shape[0] == 0:
        print(f'No data for patient {administration_row[1]["pat_nr"]}, at {administration_row[1]["drug_start"]}')
        continue
        
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    sns.lineplot(x='rounded_relative_datetime', y='ptio2', data=patient_ptio2_df, ax=ax, label='PtiO2', legend=False)
    # create second y axis for cpp values
    ax2 = ax.twinx()
    sns.lineplot(x='rounded_relative_datetime', y='cpp', data=patient_cpp_df, ax=ax2, color='magenta', label='CPP', legend=False)
    
    ax.set_xlabel('Relative time from administration (h)')
    ax.set_ylabel('PtiO2 (mmHg)')
    ax2.set_ylabel('CPP (mmHg)')
    
    # ax2.set_ylim(60,100)
    ax2.grid(None)
    
    # show legend
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(lines + lines2, labels + labels2, loc='upper right')
    
    # # add vertical line at x=0 with text label with "Diclofenac administration"
    # ax.axvline(x=0, ymin=0.015, ymax=0.1, color='black', linestyle='--', linewidth=0.7)
    # ax.text(0.5, 0.12, 'Diclofenac 75mg', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    # 
    plt.title(f'Patient {administration_row[1]["pat_nr"]} at {administration_row[1]["drug_start"]}')
    plt.show()
    

In [ ]:
# TODO: add temperature / heart rate / fio2 / curve